In [11]:
import pandas as pd
df = pd.read_pickle('/Users/buruiryanc9/Desktop/Columbia/RA/Dataset/parsed_articles.pkl')
df.head()

,title,publisher,date,section,length,body,correction_appended,load_date_at_end,source_file
0,Fleece of the Century; On Boxing,The New York Times,2017-08-27,SPORTS,1958 words,\n\nLAS VEGAS — The world finally had the two ...,False,False,NYT/22.DOCX
1,Ruling Adds to the Risk Of a Monopoly on Power,The New York Times,2019-06-28,Section A; Column 0; National Desk; Pg. 14,1472 words,\n\nAt some point or another over the last dec...,False,False,NYT/22.DOCX
2,Gerrymandering and the Rising Risk of a Monopo...,The New York Times,2019-06-28,UPSHOT,1483 words,\n\nAt some point or another over the last dec...,False,False,NYT/22.DOCX
3,How Republicans Can Make a Deal on Health Care...,The New York Times,2017-10-27,OPINION,945 words,\n\nA consensus is emerging on Capitol Hill ab...,False,False,NYT/22.DOCX
4,Harvard Suit Shows Edge Given to Elite In Recr...,The New York Times,2018-11-03,Section A; Column 0; National Desk; Pg. 16,1245 words,"\n\nBOSTON -- For the past three weeks, a succ...",False,False,NYT/22.DOCX


In [13]:
import re

def remove_load_date(text):
    if pd.isna(text):
        return text

    cleaned_text = re.sub(r'\n?Load-Date:.*$', '', text, flags=re.IGNORECASE | re.MULTILINE)
    
    return cleaned_text.strip()

df['body'] = df['body'].apply(remove_load_date)

In [16]:
print(df.iloc[2]['body'])


At some point or another over the last decade, Democrats have won the most votes but lost national elections for the presidency, the House and the Senate.
Partisan gerrymandering is just one of the reasons the Democrats are at such a disadvantage. But the Supreme Court’s decision on gerrymandering Thursday came as long-term political and demographic trends threaten to put Democrats at an even greater disadvantage in the Senate and perhaps also the presidency.
It’s even possible to imagine a future in which Republicans could effectively claim a monopoly on federal power despite continued weakness in the national vote.
Sustained minority rule — within the bounds of the Constitution — is not an imminent peril. After all, Democrats recaptured the House in November despite partisan gerrymandering. But the risk is real, and even if it does not materialize it might strain American democracy.
So on one hand, the ruling — which said federal courts can’t bar partisan gerrymandering — merely pres

In [18]:
df_filtered = df[~df['title'].str.lower().isin(['movie guide'])]

print(f"the number of artictle before filtering out: {len(df)}")
print(f"the number of artictle after filtering out: {len(df_filtered)}")

the number of artictle before filtering out: 43666
the number of artictle after filtering out: 43496


In [22]:
best_sellers = df[df['title'].str.startswith('Best Sellers:', na=False)]
print(f"Found {len(best_sellers)} 'Best Sellers:' articles")

print("\nBest Sellers article samples:")
best_sellers[['title', 'publisher', 'section', 'length']].head(10)

Found 225 'Best Sellers:' articles

Best Sellers article samples:


,title,publisher,section,length
6,Best Sellers: Combined Print & E-Book Nonficti...,The New York Times,Section BR; Column 0; Book Review Desk; Pg.,621 words
9,"Best Sellers: Hardcover Nonfiction: Sunday, Au...",The New York Times,Section BR; Column 0; Book Review Desk; Pg.,588 words
20,Best Sellers: Combined Print & E-Book Nonficti...,The New York Times,Section BR; Column 0; Book Review Desk; Pg.,581 words
22,"Best Sellers: Hardcover Nonfiction: Sunday, Au...",The New York Times,Section BR; Column 0; Book Review Desk; Pg.,583 words
32,Best Sellers: Combined Print & E-Book Nonficti...,The New York Times,Section BR; Column 0; Book Review Desk; Pg.,582 words
151,"Best Sellers: Hardcover Nonfiction: Sunday, Ja...",The New York Times,Section BR; Column 0; Book Review Desk; Pg.,638 words
153,Best Sellers: Combined Print & E-Book Nonficti...,The New York Times,Section BR; Column 0; Book Review Desk; Pg.,655 words
171,Best Sellers: Combined Print & E-Book Nonficti...,The New York Times,Section BR; Column 0; Book Review Desk; Pg.,663 words
177,"Best Sellers: Hardcover Nonfiction: Sunday, Ja...",The New York Times,Section BR; Column 0; Book Review Desk; Pg.,633 words
193,Best Sellers: Combined Print & E-Book Nonficti...,The New York Times,Section BR; Column 0; Book Review Desk; Pg.,628 words


In [24]:
print(f"\nPublisher distribution:")
best_sellers['publisher'].value_counts()


Publisher distribution:


publisher
The New York Times    225
Name: count, dtype: int64

In [23]:
print(f"\nSection distribution:")
best_sellers['section'].value_counts()


Section distribution:


section
Section BR; Column 0; Book Review Desk; Pg.       141
Section BR; Column 0; Book Review Desk             80
Section BR; Column 0; Book Review Desk; Pg. 22      2
Section BR; Column 0; Book Review Desk; Pg. 26      1
Section BR; Column 0; Book Review Desk; Pg. 99      1
Name: count, dtype: int64

In [25]:
import re

def identify_topic_specific_titles(df):
    """
    Identify all titles following "topic:specific_info" pattern
    """
    pattern = r'^[A-Za-z\s]+:\s+'
    
    topic_articles = df[df['title'].str.match(pattern, na=False)]
    return topic_articles

topic_articles = identify_topic_specific_titles(df)
print(f"Found {len(topic_articles)} articles with 'topic:info' format")

topics = topic_articles['title'].str.extract(r'^([^:]+):')[0].value_counts()
print("\nTopic type distribution:")
print(topics.head(15))

Found 2880 articles with 'topic:info' format

Topic type distribution:
0
Best Sellers           225
Review                 120
NEIGHBORHOOD REPORT     92
The Listings            63
Transcript              51
Four stars              31
STAGE                   29
THEATER                 26
POLITICS                25
Commentary              20
Kevin Riordan           17
Editorial               15
THE WAY WE LIVE NOW     15
Movies                  15
FILM                    15
Name: count, dtype: int64


In [29]:
def show_topic_samples(df, topic_articles, num_samples=2):
    # Get topic types
    topic_articles = topic_articles.copy()
    topic_articles['topic_type'] = topic_articles['title'].str.extract(r'^([^:]+):')[0]
    top_topics = topic_articles['topic_type'].value_counts().head(15)
    
    for topic_name, count in top_topics.items():
        print(f"\n{'='*60}")
        print(f"TOPIC: {topic_name} ({count} articles)")
        print(f"{'='*60}")
        
        topic_samples = topic_articles[topic_articles['topic_type'] == topic_name].head(num_samples)
        
        for idx, (_, article) in enumerate(topic_samples.iterrows(), 1):
            print(f"\nSample {idx}:")
            print(f"Title: {article['title']}")
            print(f"Publisher: {article['publisher']}")
            print(f"Date: {article['date']}")
            print(f"Section: {article['section']}")
            print(f"Length: {article['length']} words")
            
            body_preview = str(article['body'])[:10000] + "..." if len(str(article['body'])) > 200 else str(article['body'])
            print(f"Body preview: {body_preview}")
            
            working_class_keywords = ['working-class', 'working class', 'blue collar', 'labor']
            found_keywords = []
            for keyword in working_class_keywords:
                if keyword.lower() in str(article['body']).lower():
                    found_keywords.append(keyword)
            
            if found_keywords:
                print(f"Working-class keywords found: {', '.join(found_keywords)}")
            
            print("-" * 40)

show_topic_samples(df, topic_articles)


TOPIC: Best Sellers (225 articles)

Sample 1:
Title: Best Sellers: Combined Print & E-Book Nonfiction: Sunday, August 28th 2016
Publisher: The New York Times
Date: 2016-08-28
Section: Section BR; Column 0; Book Review Desk; Pg. 
Length: 621 words words
Body preview: About the Best Sellers: 
These lists are an expanded version of those appearing in the August 28, 2016 print edition of the Book Review, reflecting sales for the week ending August 13, 2016. An asterisk (*) indicates that a book's sales are barely distinguishable from those of the book above it. A dagger (?) indicates that some retailers report receiving bulk orders. For an explanation of our methodology, visit http://www.nytimes.com/best-sellers....
----------------------------------------

Sample 2:
Title: Best Sellers: Hardcover Nonfiction: Sunday, August 28th 2016
Publisher: The New York Times
Date: 2016-08-28
Section: Section BR; Column 0; Book Review Desk; Pg. 
Length: 588 words words
Body preview: About the Best Sel

In [36]:
def analyze_working_class_presence(articles_subset, subset_name):
    print(f"\n=== {subset_name} Analysis ===")
    
    working_class_keywords = ['working-class', 'working class', 'blue collar', 'labor', 'workers']
    keyword_locations = {}
    
    for keyword in working_class_keywords:
        locations = {}

        for field in ['title', 'body', 'section']:
            if field in articles_subset.columns:
                matches = articles_subset[field].str.contains(keyword, case=False, na=False).sum()
                if matches > 0:
                    locations[field] = matches
        
        if locations:
            keyword_locations[keyword] = locations
    
    if keyword_locations:
        print("Working-class keywords found:")
        for keyword, locations in keyword_locations.items():
            location_str = ", ".join([f"{field}: {count}" for field, count in locations.items()])
            print(f"  '{keyword}' - {location_str}")
    else:
        print("No working-class keywords found")
    
    valid_lengths = articles_subset['length'].dropna()
    if len(valid_lengths) > 0:
        print(f"Average length: {valid_lengths.mean():.0f} words")
        print(f"Length range: {valid_lengths.min():.0f}-{valid_lengths.max():.0f} words")
    else:
        print("No valid length data available")
    
    return keyword_locations

def identify_topic_specific_titles(df):
    pattern = r'^[A-Za-z\s]+:\s+'
    topic_articles = df[df['title'].str.match(pattern, na=False)]
    return topic_articles

topic_articles = identify_topic_specific_titles(df)
topic_articles_copy = topic_articles.copy()
topic_articles_copy['topic_type'] = topic_articles_copy['title'].str.extract(r'^([^:]+):')[0]

best_sellers = df[df['title'].str.startswith('Best Sellers:', na=False)]
best_sellers_keywords = analyze_working_class_presence(best_sellers, "Best Sellers Articles")

other_topics = topic_articles[~topic_articles['title'].str.startswith('Best Sellers:', na=False)]
if len(other_topics) > 0:
    other_keywords = analyze_working_class_presence(other_topics, "Other Topic Articles")

top_topics = topic_articles_copy['topic_type'].value_counts().head(10)

print(f"\n{'='*60}")
print("DETAILED ANALYSIS BY TOPIC TYPE")
print(f"{'='*60}")

for topic_name, count in top_topics.items():
    topic_subset = topic_articles_copy[topic_articles_copy['topic_type'] == topic_name]
    analyze_working_class_presence(topic_subset, f"{topic_name} ({count} articles)")


=== Best Sellers Articles Analysis ===
No working-class keywords found
Average length: 513 words
Length range: 420-717 words

=== Other Topic Articles Analysis ===
Working-class keywords found:
  'working-class' - title: 18, body: 2075
  'working class' - title: 17, body: 698
  'blue collar' - title: 1, body: 24
  'labor' - title: 17, body: 636
  'workers' - title: 16, body: 549
Average length: 1570 words
Length range: 1-22316 words

DETAILED ANALYSIS BY TOPIC TYPE

=== Best Sellers (225 articles) Analysis ===
No working-class keywords found
Average length: 513 words
Length range: 420-717 words

=== Review (120 articles) Analysis ===
Working-class keywords found:
  'working-class' - title: 5, body: 105
  'working class' - body: 17
  'blue collar' - body: 1
  'labor' - title: 2, body: 23
  'workers' - body: 9
Average length: 724 words
Length range: 265-1495 words

=== NEIGHBORHOOD REPORT (92 articles) Analysis ===
Working-class keywords found:
  'working-class' - title: 1, body: 79
  '

In [41]:
def analyze_keyword_context_detailed(articles_subset, subset_name):

    print(f"\n=== {subset_name} - Detailed Keyword Context Analysis ===")
    
    working_class_keywords = ['working-class', 'working class', 'blue collar', 'labor', 'workers']
    
    for keyword in working_class_keywords:
        print(f"\nKeyword: '{keyword}'")
        
        mask = articles_subset['body'].str.contains(keyword, case=False, na=False)
        keyword_articles = articles_subset[mask]
        
        if len(keyword_articles) > 0:
            print(f"Found in {len(keyword_articles)} articles")
            
            for idx, (_, article) in enumerate(keyword_articles.head(3).iterrows()):
                body = str(article['body'])
                
                import re
                sentences = re.split(r'[.!?]+', body)
                
                relevant_sentences = []
                for sentence in sentences:
                    if keyword.lower() in sentence.lower():
                        sentence = sentence.strip()
                        if len(sentence) > 20: 
                            relevant_sentences.append(sentence)
                
                if relevant_sentences:
                    print(f"\nArticle {idx+1}: {article['title']}")
                    print(f"Publisher: {article['publisher']}, Section: {article['section']}")
                    print("Context sentences:")
                    
                    for sent_idx, sentence in enumerate(relevant_sentences[:3]):
                        if len(sentence) > 400:
                            displayed_sentence = sentence[:400] + "..."
                        else:
                            displayed_sentence = sentence
                        
                        print(f"  {sent_idx+1}. {displayed_sentence}")
                    
                    if len(relevant_sentences) > 3:
                        print(f"  ... and {len(relevant_sentences)-3} more sentences with '{keyword}'")
                    
                    print("-" * 60)
        else:
            print("No articles found")

print("DETAILED KEYWORD CONTEXT ANALYSIS - LONGER EXAMPLES")
print("="*80)

topics_with_keywords = [
    'Review',                
    'NEIGHBORHOOD REPORT',   
    'The Listings',          
    'Transcript',            
    'Four stars',            
    'STAGE',                 
    'THEATER',               
    'POLITICS',              
    'Commentary'             
]

for topic_name in topics_with_keywords:
    if topic_name in topic_articles_copy['topic_type'].values:
        topic_subset = topic_articles_copy[topic_articles_copy['topic_type'] == topic_name]
        analyze_keyword_context_detailed(topic_subset, f"{topic_name} Articles")

DETAILED KEYWORD CONTEXT ANALYSIS - LONGER EXAMPLES

=== Review Articles - Detailed Keyword Context Analysis ===

Keyword: 'working-class'
Found in 105 articles

Article 1: Review: Great Pretenders Pocket Laughs in ‘The Nap’
Publisher: The New York Times , Section: THEATER
Context sentences:
  1. Bean’s story of a young working-class phenom from Sheffield and the criminal friends and relations who love (and nearly destroy) him
------------------------------------------------------------

Article 2: Review: ‘The Dark Dark,’ Beguiling Tales of Women in Metamorphosis; Books of The Times
Publisher: The New York Times , Section: BOOKS
Context sentences:
  1. Samantha Hunt’s three novels read like fables deeply rooted in reality: A young woman in a working-class seaside town believes that she is a mermaid (“The Seas”); Nikola Tesla nears his death and thinks back on his life while staying at the Hotel New Yorker (“The Invention of Everything Else”); and a woman who refuses to speak leads her

In [43]:
def calculate_standardized_relevance_scores(articles_subset, topic_name):
    scores = []
    
    for _, article in articles_subset.iterrows():
        raw_score = 0
        flags = []
        
        title = str(article['title']).lower()
        body = str(article['body']).lower()
        
        # 1. Keyword presence and frequency (0-4 points)
        working_class_keywords = ['working-class', 'working class', 'blue collar', 'labor', 'workers']
        keyword_score = 0
        keyword_count = 0
        
        for keyword in working_class_keywords:
            if keyword in body:
                keyword_count += body.count(keyword.lower())
                keyword_score += 0.5  # 0.5 points per keyword type found
                flags.append(f'body_has_{keyword.replace(" ", "_")}')
            if keyword in title:
                keyword_score += 1.5  # 1.5 points for title matches
                flags.append(f'title_has_{keyword.replace(" ", "_")}')
        
        # Bonus for high keyword frequency
        if keyword_count >= 5:
            keyword_score += 1
            flags.append('very_high_keyword_frequency')
        elif keyword_count >= 3:
            keyword_score += 0.5
            flags.append('high_keyword_frequency')
        
        # Cap keyword score at 4
        keyword_score = min(keyword_score, 4)
        raw_score += keyword_score
        
        # 2. Topic relevance (0-3 points)
        if topic_name in ['POLITICS', 'Commentary']:
            raw_score += 3  # Highly likely to be relevant
            flags.append('high_relevance_topic')
        elif topic_name in ['NEIGHBORHOOD REPORT', 'Transcript']:
            raw_score += 2  # Likely relevant
            flags.append('medium_high_relevance')
        elif topic_name in ['Review']:
            raw_score += 1  # Could be relevant
            flags.append('potentially_relevant')
        elif topic_name in ['STAGE', 'THEATER']:
            raw_score += 0.5  # Neutral
            flags.append('cultural_content')
        elif topic_name in ['Four stars', 'The Listings']:
            raw_score += 0  # Less likely to be relevant
            flags.append('likely_irrelevant')
        
        # 3. Article quality indicators (0-2 points)
        length = article.get('length', 0)
        if pd.notna(length):
            if length >= 1000:
                raw_score += 1
                flags.append('substantial_article')
            elif length >= 500:
                raw_score += 0.5
                flags.append('medium_length')
            elif length < 200:
                raw_score -= 0.5
                flags.append('very_short')
        
        # 4. Additional context clues (0-1 points)
        # Check for related terms that might indicate relevance
        related_terms = ['union', 'employment', 'wages', 'workplace', 'factory', 'manufacturing', 'economy', 'jobs']
        related_count = sum(1 for term in related_terms if term in body.lower())
        if related_count >= 3:
            raw_score += 1
            flags.append('rich_related_context')
        elif related_count >= 1:
            raw_score += 0.5
            flags.append('some_related_context')
            
        standardized_score = min(raw_score, 10)  # Cap at 10
        standardized_score = max(standardized_score, 0)  # Floor at 0
        standardized_score = round(standardized_score, 1)
        
        scores.append({
            'title': article['title'],
            'topic_type': topic_name,
            'relevance_score': standardized_score,
            'keyword_count': keyword_count,
            'flags': flags,
            'length': length
        })
    
    return pd.DataFrame(scores)

print(f"{'='*70}")
print("STANDARDIZED RELEVANCE SCORING (0-10 SCALE)")
print(f"{'='*70}")
print("Score Interpretation:")
print("  8.0-10.0: Highly Relevant (definitely keep)")
print("  6.0-7.9:  Moderately Relevant (likely keep)")
print("  4.0-5.9:  Potentially Relevant (review carefully)")
print("  2.0-3.9:  Low Relevance (likely remove)")
print("  0.0-1.9:  Not Relevant (remove)")
print("="*70)

topics_with_keywords = [
    'Review', 'NEIGHBORHOOD REPORT', 'The Listings', 'Transcript',
    'Four stars', 'STAGE', 'THEATER', 'POLITICS', 'Commentary'
]

standardized_summary = {}
all_standardized_articles = []

for topic_name in topics_with_keywords:
    if topic_name in topic_articles_copy['topic_type'].values:
        topic_subset = topic_articles_copy[topic_articles_copy['topic_type'] == topic_name]
        
        print(f"\n{'='*50}")
        print(f"TOPIC: {topic_name} ({len(topic_subset)} articles)")
        print(f"{'='*50}")
        
        relevance_scores = calculate_standardized_relevance_scores(topic_subset, topic_name)
        all_standardized_articles.append(relevance_scores)
        
        print(f"Average score: {relevance_scores['relevance_score'].mean():.2f}/10")
        print(f"Score range: {relevance_scores['relevance_score'].min():.1f} - {relevance_scores['relevance_score'].max():.1f}")
        print(f"Median score: {relevance_scores['relevance_score'].median():.1f}")
    
        highly_relevant = relevance_scores[relevance_scores['relevance_score'] >= 8.0]
        moderately_relevant = relevance_scores[(relevance_scores['relevance_score'] >= 6.0) & (relevance_scores['relevance_score'] < 8.0)]
        potentially_relevant = relevance_scores[(relevance_scores['relevance_score'] >= 4.0) & (relevance_scores['relevance_score'] < 6.0)]
        low_relevance = relevance_scores[(relevance_scores['relevance_score'] >= 2.0) & (relevance_scores['relevance_score'] < 4.0)]
        not_relevant = relevance_scores[relevance_scores['relevance_score'] < 2.0]
        
        print(f"\nStandardized relevance breakdown:")
        print(f"  Highly Relevant (8.0-10.0): {len(highly_relevant)} articles ({len(highly_relevant)/len(relevance_scores)*100:.1f}%)")
        print(f"  Moderately Relevant (6.0-7.9): {len(moderately_relevant)} articles ({len(moderately_relevant)/len(relevance_scores)*100:.1f}%)")
        print(f"  Potentially Relevant (4.0-5.9): {len(potentially_relevant)} articles ({len(potentially_relevant)/len(relevance_scores)*100:.1f}%)")
        print(f"  Low Relevance (2.0-3.9): {len(low_relevance)} articles ({len(low_relevance)/len(relevance_scores)*100:.1f}%)")
        print(f"  Not Relevant (0.0-1.9): {len(not_relevant)} articles ({len(not_relevant)/len(relevance_scores)*100:.1f}%)")
    
        if len(highly_relevant) > 0:
            print(f"\nHighly Relevant examples (8.0-10.0):")
            for _, row in highly_relevant.head(2).iterrows():
                print(f"  Score {row['relevance_score']}: {row['title'][:65]}...")
                print(f"    Keywords: {row['keyword_count']}, Length: {row['length']} words")
        
        if len(moderately_relevant) > 0:
            print(f"\nModerately Relevant examples (6.0-7.9):")
            for _, row in moderately_relevant.head(2).iterrows():
                print(f"  Score {row['relevance_score']}: {row['title'][:65]}...")
        
        if len(not_relevant) > 0:
            print(f"\nNot Relevant examples (0.0-1.9):")
            for _, row in not_relevant.head(2).iterrows():
                print(f"  Score {row['relevance_score']}: {row['title'][:65]}...")
    
        standardized_summary[topic_name] = {
            'total': len(topic_subset),
            'highly_relevant': len(highly_relevant),
            'moderately_relevant': len(moderately_relevant),
            'potentially_relevant': len(potentially_relevant),
            'low_relevance': len(low_relevance),
            'not_relevant': len(not_relevant),
            'avg_score': relevance_scores['relevance_score'].mean(),
            'median_score': relevance_scores['relevance_score'].median()
        }

all_standardized_df = pd.concat(all_standardized_articles, ignore_index=True)

print(f"\n{'='*70}")
print("OVERALL STANDARDIZED SUMMARY")
print(f"{'='*70}")

print(f"Total articles analyzed: {len(all_standardized_df)}")
print(f"Overall average score: {all_standardized_df['relevance_score'].mean():.2f}/10")
print(f"Overall median score: {all_standardized_df['relevance_score'].median():.1f}/10")
print(f"Score standard deviation: {all_standardized_df['relevance_score'].std():.2f}")

overall_highly = all_standardized_df[all_standardized_df['relevance_score'] >= 8.0]
overall_moderately = all_standardized_df[(all_standardized_df['relevance_score'] >= 6.0) & (all_standardized_df['relevance_score'] < 8.0)]
overall_potentially = all_standardized_df[(all_standardized_df['relevance_score'] >= 4.0) & (all_standardized_df['relevance_score'] < 6.0)]
overall_low = all_standardized_df[(all_standardized_df['relevance_score'] >= 2.0) & (all_standardized_df['relevance_score'] < 4.0)]
overall_not = all_standardized_df[all_standardized_df['relevance_score'] < 2.0]

print(f"\nOverall distribution:")
print(f"  Highly Relevant (8.0-10.0): {len(overall_highly)} articles ({len(overall_highly)/len(all_standardized_df)*100:.1f}%)")
print(f"  Moderately Relevant (6.0-7.9): {len(overall_moderately)} articles ({len(overall_moderately)/len(all_standardized_df)*100:.1f}%)")
print(f"  Potentially Relevant (4.0-5.9): {len(overall_potentially)} articles ({len(overall_potentially)/len(all_standardized_df)*100:.1f}%)")
print(f"  Low Relevance (2.0-3.9): {len(overall_low)} articles ({len(overall_low)/len(all_standardized_df)*100:.1f}%)")
print(f"  Not Relevant (0.0-1.9): {len(overall_not)} articles ({len(overall_not)/len(all_standardized_df)*100:.1f}%)")

STANDARDIZED RELEVANCE SCORING (0-10 SCALE)
Score Interpretation:
  8.0-10.0: Highly Relevant (definitely keep)
  6.0-7.9:  Moderately Relevant (likely keep)
  4.0-5.9:  Potentially Relevant (review carefully)
  2.0-3.9:  Low Relevance (likely remove)
  0.0-1.9:  Not Relevant (remove)

TOPIC: Review (120 articles)
Average score: 2.40/10
Score range: 1.5 - 5.0
Median score: 2.0

Standardized relevance breakdown:
  Highly Relevant (8.0-10.0): 0 articles (0.0%)
  Moderately Relevant (6.0-7.9): 0 articles (0.0%)
  Potentially Relevant (4.0-5.9): 6 articles (5.0%)
  Low Relevance (2.0-3.9): 95 articles (79.2%)
  Not Relevant (0.0-1.9): 19 articles (15.8%)

Not Relevant examples (0.0-1.9):
  Score 1.5: Review: Titus Andronicus Releases a Punk Opera...
  Score 1.5: Review: War is good as 50th anniversary of ‘The World Is a Ghetto...

TOPIC: NEIGHBORHOOD REPORT (92 articles)
Average score: 2.98/10
Score range: 2.5 - 5.5
Median score: 3.0

Standardized relevance breakdown:
  Highly Relevant (8.